# MQML notebook for conductresist

In this notebook, we want to show how to setup and use the module conductresist.py. This module has an instrument called ConductResist, which holds and calculates parameters related to differential conductance and resistance.

### Basic imports and logging

In [1]:
from qcodes.station import Station
from qcodes import initialise_or_create_database_at
from qcodes.logger import start_all_logging
from mqml.instrument.conductresist import ConductResist
from qcodes.instrument.parameter import Parameter
import time
import os
start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /home/runner/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


Qcodes Logfile : /home/runner/.qcodes/logs/210126-3966-qcodes.log


### Creating qcodes station: 

For the purpose of this notebook, we create an empty qcodes station object to hold our upcoming ConductResist instruments. In a real experiment, we likely need to load a station yaml file into our created station object, which this yaml file includes all physical instruments needed for the experiment. 

In [2]:
station = Station()

# If the station.yaml file exists, load the yaml file to the station instead of the above:
# station = Station(config_file='[full path to the station]/station.yaml')

Let's look at our station snapshot:

In [3]:
station.snapshot()

{'instruments': {}, 'parameters': {}, 'components': {}, 'config': None}

### Setting up ConductResist

ConductResist calculates differential conductance and resistance using voltag and amplitude inputs of two lock-in amplifiers. These inputs are qcodes Parameters. Since we do not have access to lock-in amplifiers in this notebook, we create qcodes Parameters to run the notebook. We need 3 parameters to instantiate this class (we create these parameters with initial values to be able to get these vales when needed):

In [4]:
lockin1_X = Parameter('lockin1_X', set_cmd=None, unit='V', initial_value=1)
lockin1_amp = Parameter('lockin1_amp', set_cmd=None, unit='V', initial_value=2)
lockin2_X = Parameter('lockin2_X', set_cmd=None, unit='V', initial_value=3)

# In a real experiment, we may follow something like:
# lockin1 = station.load_instrument('the name of the first lock-in in the station.yaml, e.g., lockin1')
# lockin2 = station.load_instrument('the name of the second lock-in in the station.yaml, e.g., lockin2')
# lockin1_X = lockin1.X
# lockin1_amp = lockin1.amplitude
# lockin2_X = lockin2.X

Now, we instantiate ConductResist as below:

In [5]:
lockin_1_2 = ConductResist('lockin_1_2', lockin1_volt=lockin1_X, lockin1_amp=lockin1_amp, lockin2_volt=lockin2_X)

Then, it is possible to instantiate as many ConductResist instance as you want, if you need more than one lockin pair in your experimental setup.

In order to retrive conductance and resistance out of our created lockin_1_2, we need to set amplification and division parameters in this created lockin_1_2 instance, otherwise we will get TypeError of not setting them. Note that ACdive and DCdive are defined with initial values of 1e-4 and 1e-2, respectively. Thus, we can leave defining these two parameters, if we want to use their initial values:

In [6]:
lockin_1_2.GIamp(1e7) # Current amplification
lockin_1_2.GVamp(100) # Voltage amplification

Now, we can get the values of conductance and resistance:

In [7]:
lockin_1_2.diff_conductance_fpm()

0.04302134575919792

In [8]:
lockin_1_2.conductance_tpm()

6.453201863879687

In [9]:
lockin_1_2.resistance_fpm()

300000.0

We can add the created lockin_1_2 to our station:

In [10]:
station.add_component(lockin_1_2)

'lockin_1_2'

The snapshot of the station should show our lockin_1_2 instrument and its parameters

In [11]:
station.snapshot()

{'instruments': {'lockin_1_2': {'functions': {},
   'submodules': {},
   '__class__': 'mqml.instrument.conductresist.ConductResist',
   'parameters': {'IDN': {'__class__': 'qcodes.instrument.parameter.Parameter',
     'full_name': 'lockin_1_2_IDN',
     'value': {'vendor': None,
      'model': 'lockin_1_2',
      'serial': None,
      'firmware': None},
     'raw_value': {'vendor': None,
      'model': 'lockin_1_2',
      'serial': None,
      'firmware': None},
     'ts': '2021-01-26 14:38:55',
     'label': 'IDN',
     'unit': '',
     'name': 'IDN',
     'vals': '<Anything>',
     'inter_delay': 0,
     'instrument': 'mqml.instrument.conductresist.ConductResist',
     'instrument_name': 'lockin_1_2',
     'post_delay': 0},
    'GIamp': {'__class__': 'qcodes.instrument.parameter.Parameter',
     'full_name': 'lockin_1_2_GIamp',
     'value': 10000000.0,
     'raw_value': 10000000.0,
     'ts': '2021-01-26 14:38:55',
     'label': 'Current Amplification',
     'unit': '',
     'name':

Let's examine what happens if the current value of an argument in an instance of ConductResist is zero (in this case, lockin3_amp):

In [12]:
lockin3_X = Parameter('lockin1_X', set_cmd=None, unit='V', initial_value=1)
lockin3_amp = Parameter('lockin1_amp', set_cmd=None, unit='V', initial_value=0)
lockin4_X = Parameter('lockin2_X', set_cmd=None, unit='V', initial_value=3)

In [13]:
lockin_3_4 = ConductResist('lockin_3_4', lockin1_volt=lockin3_X, lockin1_amp=lockin3_amp, lockin2_volt=lockin4_X)

In [14]:
lockin_3_4.GIamp(1e7) # Current amplification
lockin_3_4.GVamp(100) # Voltage amplification

In [15]:
lockin_3_4.diff_conductance_fpm()

0.04302134575919792

In [16]:
lockin_3_4.conductance_tpm()

2021-01-26 14:38:55,885 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 110 ¦ /home/runner/work/MQML-scripts/MQML-scripts/mqml/instrument/conductresist.py:86: UserWarning: The denominator is zero, returning NaN
  warnings.warn('The denominator is zero, returning NaN')



nan

In [17]:
lockin_3_4.resistance_fpm()

300000.0

We see that lockin_3_4.conductance_tpm() is returning a warning and 'nan'. This is because we have the division by zero case in the calculation of the resistance_fpm as the result of the current value of lockin3_amp is zero. This designe is intentional as we want to warn the user about this occurance and return the output value as numpy nan to not stop the experiment.